In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import lxml
import requests
import psycopg2
import time

In [2]:
def extract_to_pg(page):
    # create soup 
    soup = BeautifulSoup(page, 'lxml')
    # check table
    all_tr = soup.find_all('tr')

    pg_config = "dbname=stock user=postgres password=123qwe host=127.0.0.1 port=5432"
    pg_conn = psycopg2.connect(pg_config)
    print("PostgreSQL DB connected")
    curs = pg_conn.cursor()
    #Since 3 first row is the header, data is stored on the 4th row onwards
    for j in range(3,len(all_tr)):
        #row is our j'th row
        row=all_tr[j]
        all_td = row.find_all('td')
        
        if len(all_td) != 15:
            continue

        data_row = []
        for td in all_td:
            data = td.text
            data = data.replace("\xa0", "").replace(",", "")
            if "(" in data:
                data = data.split("(")[0]
            try:
                if "." in data:
                    data = float(data)
                else:
                    data = int(data)
            except:
                pass
            data_row.append(data)
        date_c = data_row[0]
        open_price = int(data_row[9]*1000)
        close_price = int(data_row[2]*1000)
        highest_price = int(data_row[10]*1000)
        lowest_price = int(data_row[11]*1000)
        change = data_row[3]
        ex_match = data_row[5]
        ex_value = data_row[6]
        deal_match = data_row[7]
        deal_value = data_row[8]
        value = ( date_c, open_price, close_price, highest_price,
                lowest_price, change, ex_match, ex_value, 
                deal_match, deal_value)
        curs.execute("INSERT INTO fpt VALUES {};".format(value))
    pg_conn.commit()
    print("Insert all rows")
    pg_conn.close()        
    return

In [4]:
# Get the page with selenium driver
url='https://s.cafef.vn/Lich-su-giao-dich-FPT-1.chn'
chrome_options = webdriver.ChromeOptions() 
chrome_options.add_argument("start-maximized")
#driver = webdriver.Chrome()
driver = webdriver.Chrome(options=chrome_options, executable_path=r'./chromedriver.exe')
driver.get(url)
page = driver.page_source
#soup = BeautifulSoup(page, 'lxml')


In [9]:
#extract_to_pg(page)

PostgreSQL DB connected
Insert all rows


In [5]:
count = 149
time_wait = 0
import time
while True:
    try:
        # Check the next button before extract data
        title_string = " Next to Page {}".format(count+1)
        driver.find_element_by_xpath('.//a[@title="{}"]'.format(title_string))
        print("Finish loading page {}".format(count))
        # Extract data
        page = driver.page_source
        extract_to_pg(page)
        print("Finish extracting page number {}".format(count))
        
        # Click to next page
        #driver.find_element_by_xpath('.//a[@title=" Next to Page 2"]').click()
        #driver.find_element_by_xpath('.//a[contains(@title, " Next to Page")]').click()
        #driver.find_element_by_xpath('.//a[@title="{}"]'.format(title_string)).click()
        
        txt = "'ctl00$ContentPlaceHolder1$ctl03$pager2','{}'".format(count+1)
        driver.find_element_by_xpath('.//a[contains(@href, "{}")]'.format(txt)).click() 
        count += 1
        print("Go to page {}".format(count))
        time.sleep(10)
        time_wait = 0
          
    except :
        if time_wait > 100:
            print("No more pages or timeout")
            confirm = input("Continue? Manually navigate to the page {} then type yes: ".format(count))
            if confirm != "yes" :
                break 
            else:
                time_wait = 0
        print("Oops!!! Something wrong! Waiting ...")
        time_wait += 10
        time.sleep(10)
        
print("Num of pages: ", count)


Oops!!! Something wrong! Waiting ...
Oops!!! Something wrong! Waiting ...
Oops!!! Something wrong! Waiting ...
Finish loading page 149
PostgreSQL DB connected
Insert all rows
Finish extracting page number 149
Go to page 150
Finish loading page 150
PostgreSQL DB connected
Insert all rows
Finish extracting page number 150
Go to page 151
Finish loading page 151
PostgreSQL DB connected
Insert all rows
Finish extracting page number 151
Go to page 152
Finish loading page 152
PostgreSQL DB connected
Insert all rows
Finish extracting page number 152
Go to page 153
Finish loading page 153
PostgreSQL DB connected
Insert all rows
Finish extracting page number 153
Go to page 154
Finish loading page 154
PostgreSQL DB connected
Insert all rows
Finish extracting page number 154
Go to page 155
Finish loading page 155
PostgreSQL DB connected
Insert all rows
Finish extracting page number 155
Go to page 156
Finish loading page 156
PostgreSQL DB connected
Insert all rows
Finish extracting page number 156
